# Introduction

The Lab. 1 is an introduction to the laboratories environment for the subject Foundation Models. 

# Kaggle environment

A tour through Kaggle will be shown during the first session.

# Local Environment

It is possible to work directly on Kaggle but it is recommended to  create a local environment and then you can use Visual Studio Code to work with the notebooks. The `local_requirements.txt` contains the requirements for the local environment, and uses the same packages version that Kaggle.

Please, create a local environment (`F1 -> Python Create Environment...`) and apply the requirements:
```bash
pip install -r local_requirements.txt
```

Please, select Python 3.12 to keep compatibility with Kaggle.

> **Note** The default installation uses Torch only compatible with CPU. If you are using a Mac with Apple silicon, `mps` will be used automatically. For Linux or Windows/WSL, it is recommended to install CUDA 12.8 and the correspondent version of torch. If you do not have GPU, just run on local the simple examples and use Kaggle for the heavy workload (e.g. fine-tunning, inference with huge models)

# Use your first model

In [ ]:
import torch
from transformers import pipeline

model_id = "microsoft/Phi-3-mini-128k-instruct"

pipe = pipeline("text-generation", model_id)

pipe("Hello, world!", max_new_tokens=100)

## Transformers

The *transformers* library by [Hugging Face](https://huggingface.co) is a Python package that provides state-of-the-art implementations of Transformer-based models for natural language processing (NLP), computer vision, and multimodal tasks. It allows developers and researchers to load pretrained models, fine-tune them on custom datasets, and deploy them for inference. The library supports a wide range of Transformer architectures, including BERT, GPT, RoBERTa, T5, and many others, covering both encoder-only, decoder-only, and encoder-decoder models.

One of the key features of *transformers* is its high-level API for text, image, and audio tasks. With just a few lines of code, users can perform text generation, classification, question answering, summarization, translation, and token-level tasks. The library also integrates seamlessly with popular deep learning frameworks such as PyTorch and TensorFlow, offering both flexibility and performance.

In addition to model implementations, *transformers* provides tokenizers, which handle preprocessing text into tokens suitable for the models, supporting techniques such as WordPiece, Byte-Pair Encoding (BPE), and SentencePiece. The library also includes training utilities, pipelines, and ready-to-use pretrained models through the Hugging Face Model Hub, making it possible to leverage zero-shot, few-shot, and transfer learning without building models from scratch.

Overall, the *transformers* library is designed to accelerate NLP research and application development by providing reliable, scalable, and easy-to-use implementations of modern Transformer models. It has become a standard tool in the AI ecosystem for anyone working with language, text, or multimodal data.

## Models

Each model has an entry at Hugging Face. For the microsoft Phi-3 model you can check the information in [microsoft/Phi-3-mini-128k-instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct).

The most interesting is the Model Card, were you can read informationa about the model and how to use it.

### Model name

Model names folows a convention to describe them. For example:

```
microsoft / Phi - 3 - mini - 128k - instruct
    |       |     |    |      |      |
   Org   Family  Ver. Size Context  Task
```
Where:
 * Org is the organization providing the model, in this case Microsoft
 * Family represents the model family or architecture
 * Version states the version of the model
 * Size is the scale of the model (small, medium, etc.)
 * Context is the size of tokens for the context
 * Task is how the model was trained (in this case, to follow instructions) 


## pipeline

*pipeline* is an easy way to use models for quick inference, as it abstracts most of the complexity code from transformers library.

# Chat format

One useful information that use to be in the Model Card is the Chat format. It is important to follow it, as the model was fine-tunned (instructed in our case) with that format. For example, for Phi-3 we have:

In [ ]:
prompt = """<|system|>
You are a helpful travel assistant.<|end|>
<|user|>
I am going to Paris, what should I see?<|end|>
<|assistant|>"""

answer = pipe(prompt, max_new_tokens=500)
print(answer)

And we can parse the answer:

In [ ]:
model_answer = answer[0]['generated_text'].split("<|assistant|>")[1]
print(model_answer)

Each model has its own way of formatting the instructions and outputs, but this is also managed by `transformers`. Let's define the prompt in a transformer's standard way:

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful travel assistant."},
    {"role": "user", "content": "I am going to Paris, what should I see?"}
]
output = pipe(messages, max_new_tokens=500)
print(output[0]['generated_text']) 

In general, we have:
 * System role: General instructions where are defined the expectations and for example, the input output format, the kind of the task
 * User role: Contains the task itself, a question in our example
 * Assistant role: model output

The messages can be concatenated, and are the basis for a context and/or few-shots prompting. For example:

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful travel assistant.'},
    {'role': 'user', 'content': 'I am going to Paris, what should I see?'},
    {'role': 'assistant', 'content': " Paris is a city that offers an abundance of attractions, each with its own unique charm. Here's a list of must-see places that will allow you to experience the essence of Paris:\n\n1. Eiffel Tower: Start your trip with a visit to the iconic Eiffel Tower. Take the elevator to the top for a breathtaking panoramic view of the city.\n\n2. Louvre Museum: Home to the famous Mona Lisa and thousands of other masterpieces, the Louvre is a must-visit for art lovers.\n\n3. Notre-Dame Cathedral: Although currently closed for renovations, the Cathedral is an architectural marvel and will be reopened soon. It's still worth taking a walk around the area to admire its beauty.\n\n4. Arc de Triomphe: Located at the western end of the Champs-Élysées, the Arc de Triomphe honors those who fought and died for France. The view from the top is spectacular during sunset.\n\n5. Montmartre: This charming neighborhood offers a glimpse into Paris's bohemian past and is known for the Sacré-Cœur Basilica and the vibrant street art.\n\n6. Palace of Versailles: A short train ride from central Paris, the Palace of Versailles is a grandiose symbol of absolute monarchy. Explore its opulent rooms and gardens.\n\n7. Seine River Cruise: See Paris's most famous landmarks, including the Eiffel Tower, Notre-Dame, and Louvre, from the serene waters of the Seine River.\n\n8. Latin Quarter: This historic district is famous for its bohemian atmosphere, lively cafes, and quaint bookshops.\n\n9. Champs-Élysées: Take a stroll down this iconic avenue, which is lined with theaters, cafes, and luxury shops.\n\n10. Musée d'Orsay: Housed in a former railway station, this museum showcases an extensive collection of Impressionist and Post-Impressionist masterpieces.\n\n11. Sainte-Chapelle: Known for its st"},
    {'role': 'user', "content": "Interesting, could you tell me more about Notre Dame Cathedral?"}
]
output = pipe(messages, max_new_tokens=500)
print(output[0]['generated_text']) 

# Assignment #1

The first assignment is to build a Q&A system, using the methods we described above. The requirements are:
 * Keep it simple: we just need a function to send a query and get a response.
 * Keep the context in all interactions (> Hint: `output[0]['generated_text]`can be used to create the next input)
 * Test you system with several related questions about the topic of your interest

# Working with Kaggle

## Integration with Kaggle

If the local environment is created correctly, you will have the command `kaggle`, which allows you to interact with Kaggle:

```bash
% kaggle -h
usage: kaggle [-h] [-v] [-W] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set, unset}
    competitions (c)    Commands related to Kaggle competitions
    datasets (d)        Commands related to Kaggle datasets
    kernels (k)         Commands related to Kaggle kernels
    models (m)          Commands related to Kaggle models
    files (f)           Commands related files
    config              Configuration settings
    ```

You will need to authorize the access of the tool to the Kaggle's API. For that, go to your [kaggle's account](https://www.kaggle.com/settings/account) and create a **Legacy API Credential**.

![Legacy API Credentials](./img/lab1/legacy_api.png "Legacy API Credentials")

Move the downloaded file ('kaggle.json') to a directory '.kaggle' in your home:
```bash
(.venv)  ~ % mkdir ~/.kaggle/
(.venv)  ~ % mv ~/Downloads/kaggle.json ~/.kaggle 
(.venv)  ~ % chmod 600 ~/.kaggle/kaggle.json
(.venv)  ~ % kaggle competitions list   
ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3    2026-04-15 23:59:00  Featured         2,207,152 Usd       1535            True  
https://www.kaggle.com/competitions/vesuvius-challenge-surface-detection         2026-02-13 23:59:00  Research           200,000 Usd        734           False  
https://www.kaggle.com/competitions/med-gemma-impact-challenge                   2026-02-24 23:59:00  Featured           100,000 Usd         51           False  
https://www.kaggle.com/competitions/csiro-biomass                                2026-01-28 23:59:00  Research            75,000 Usd       3757           False  
https://www.kaggle.com/competitions/stanford-rna-3d-folding-2                    2026-03-25 23:59:00  Featured            75,000 Usd        448           False  
https://www.kaggle.com/competitions/santa-2025                                   2026-01-30 23:59:00  Featured            50,000 Usd       3281           False  
...
```

## Upload and run a kernel

If you prefer to work on local, it is possible to upload and run a notebook from your computer. 

First, from the directory where you have the notebook, `init`a new notebook:
```bash
(.venv) src % kaggle kernels init
Kernel metadata template written to: ./VisualStudioProjects/unie-foundation-models/src/kernel-metadata.json
```
The commands creates a new metadata for a kernel. Edit it, and put the information of the `Lab 1` notebook:
```json
{
  "id": "<put your kaggle user here>/introduction",
  "title": "Introduction",
  "code_file": "Lab 1. Introduction.ipynb",
  "language": "python",
  "kernel_type": "notebook",
  "is_private": "true",
  "enable_gpu": "true",
  "enable_tpu": "false",
  "enable_internet": "true",
  "dataset_sources": [],
  "competition_sources": [],
  "kernel_sources": [],
  "model_sources": []
}
```



> Note: Please check the GPU quota when you activate it, it is limited and maybe you will need it for other labs!

You can upload and run the kernel using push option:
```bash
(.venv) src % kaggle kernels push 
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/marcmolla/introduction
```

To check the status of the notebook:
```bash
(.venv) src % kaggle kernels status introduction
introduction has status "KernelWorkerStatus.RUNNING"
```

You can also go to the notebook link and see the Logs:
![Logs](./img/lab1/logs.png "Logs")

After a while (around 3 minutes), you will have the results in the `Notebook` section:
![Results](./img/lab1/results.png "Results")

## Download kernel and logs

>Note: Be careful, this will override your work with current kaggle's version!

```bash
(.venv) src % kaggle kernels output 1-introduction
Kernel log downloaded to ./unie-foundation-models/src/1-introduction.log 
(.venv) src % kaggle kernels pull 1-introduction  
Source code downloaded to ./unie-foundation-models/src/Lab 1. Introduction.ipynb
```